:::{canonical-tutorial} tutorials/experimental/dask
:::


# Using `dask` with Scanpy


:::{warning} 🔪 **Beware sharp edges!** 🔪

`dask` support in `scanpy` is new and highly experimental!

Many functions in `scanpy` **do not** support `dask` and may exhibit unexpected behaviour if dask arrays are passed to them. Stick to what's outlined in this tutorial and you should be fine!

Please report any issues you run into over on the issue tracker.
:::


[`dask`](https://www.dask.org) is a popular out-of-core, distributed array processing library that scanpy is beginning to support. Here we walk through a quick tutorial of using `dask` in a simple analysis task.


This notebook relies on optional dependencies in dask, `sklearn-ann` and `annoy`. Install them with:

```sh
pip install -U "scanpy[dask]" "dask[distributed,diagnostics]" sklearn-ann annoy
```

(`scanpy[dask]` means to install scanpy together with `dask[array]`, but will also always make sure that a compatible dask version is selected)


In [ ]:
from pathlib import Path

import dask.distributed as dd
import scanpy as sc
import anndata as ad
import h5py

sc.logging.print_header()

Here, we'll be working with a moderately large dataset of 1.4 million cells taken from: [COVID-19 immune features revealed by a large-scale single-cell transcriptome atlas](https://cellxgene.cziscience.com/collections/0a839c4b-10d0-4d64-9272-684c49a2c8ba)


In [2]:
def download(url: str, path: Path) -> None:
    """Download a file from `url` and save it to `path`, showing a progress bar."""
    from tqdm.autonotebook import tqdm
    from urllib.request import urlretrieve

    pb = tqdm(unit="B", unit_scale=True, unit_divisor=1024)

    def update(b: int = 1, bsize: int = 1, tsize: int | None = None):
        if tsize is not None:
            pb.total = tsize
        return pb.update(b * bsize - pb.n)

    try:
        with pb:
            urlretrieve(url, path, reporthook=update)
    except BaseException:
        path.unlink(missing_ok=True)
        raise

In [3]:
cell_atlas_path = Path("data/cell_atlas.h5ad")
cell_atlas_path.parent.mkdir(exist_ok=True)
if not cell_atlas_path.exists():
    download(
        "https://datasets.cellxgene.cziscience.com/82eac9c1-485f-4e21-ab21-8510823d4f6e.h5ad",
        cell_atlas_path,
    )

For more information on using distributed computing via `dask`, please see their [documentation](https://docs.dask.org/en/stable/deploying-python.html). In short, one needs to define both a cluster and a client to have some degree of control over the compute resources dask will use. It's very likely you will have to tune the number of workers and amount of memory per worker along with your chunk sizes.


In [4]:
cluster = dd.LocalCluster(n_workers=3)
client = dd.Client(cluster)

:::{note}
In this notebook we will be demonstrating some computations in scanpy that use `scipy.sparse` classes within each dask chunk. Be aware that this is currently poorly supported by `dask`, and that if you want to interact with the `dask` arrays in any way other than though the `anndata` and `scanpy` libraries you will likely need to densify each chunk.

All operations in `scanpy` and `anndata` that work with sparse chunks also work with dense chunks.

The advantage of using sparse chunks are:

- The ability to work with fewer, larger chunks
- Accelerated computations per chunk (e.g. don't need to `sum` all those extra zeros)

You can convert from `sparse` to `dense` chunks via:

```python
X = X.map_blocks(lambda x: x.toarray(), dtype=X.dtype, meta=np.array([]))
```

And in reverse:

```python
X = X.map_blocks(sparse.csr_matrix)
```

Note that you will likely have to work with smaller chunks when doing this, via a rechunking operation. We suggest using a factor of the larger chunk size to achieve the most efficient rechunking.

:::


In [5]:
SPARSE_CHUNK_SIZE = 100_000

Dask provides extensive tooling for monitoring your computation. You can access that via the dashboard started when using any of their distributed clusters.


In [ ]:
client

We'll convert the `X` representation to `dask` using `anndata.experimental.read_elem_as_dask`.


The file we've retrieved from cellxgene has already been processed. Since this tutorial is demonstrating processing from counts, we're just going to access the counts matrix and annotations.


In [ ]:
%%time
with h5py.File(cell_atlas_path, "r") as f:
    adata = ad.AnnData(
        obs=ad.io.read_elem(f["obs"]),
        var=ad.io.read_elem(f["var"]),
    )
    adata.X = ad.experimental.read_elem_as_dask(
        f["raw/X"], chunks=(SPARSE_CHUNK_SIZE, adata.shape[1])
    )

We've optimized a number of scanpy functions to be completely lazy. That means it will look like nothing is computed when you call an operation on a dask array, but only later when you hit compute.

In some cases it's currently unavoidable to skip all computation, and these cases will kick off compute for all the delayed operations immediately.


In [ ]:
%%time
adata.layers["counts"] = adata.X.copy()  # Making sure we keep access to the raw counts
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
%%time
sc.pp.log1p(adata)

Highly variable genes needs to add entries into `obs`, which currently does not support lazy column. So computation will occur immediately on call.


In [ ]:
%%time
sc.pp.highly_variable_genes(adata)

Scanpy 1.11+ supports computing PCA on `dask` arrays with sparse chunks.


In [ ]:
%%time
sc.pp.pca(adata)

While most of the PCA computation runs immediately, the last step (computing the observation loadings) is lazy, so must be triggered manually to avoid recomputation.


In [ ]:
%%time
adata.obsm["X_pca"] = adata.obsm["X_pca"].compute()

In [ ]:
adata

Now that we've computed our PCA let's take a look at it:


In [ ]:
sc.pl.pca(adata, color="majorType")

Further support for `dask` is a work in progress. However, many operations past this point can work with the dimensionality reduction directly in memory. With scanpy `1.10` many of these operations can be accelerated to make working with large datasets significantly easier. For example:

- Using alternative KNN backends for faster neighbor calculation {doc}`/how-to/knn-transformers`
- Using the `igraph` backend for clustering


In [ ]:
%%time
from sklearn_ann.kneighbors.annoy import AnnoyTransformer  # noqa: E402

transformer = AnnoyTransformer(n_neighbors=15)
sc.pp.neighbors(adata, transformer=transformer)

In [ ]:
%%time
sc.tl.leiden(adata, flavor="igraph", n_iterations=2)

UMAP computation can still be rather slow, taking longer than the rest of this notebook combined:


In [ ]:
%%time
sc.tl.umap(adata)
sc.pl.umap(adata, color=["leiden", "majorType"])